# Predicting Sex Assigned at Birth with Regression

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import pandas as pd
import numpy as np
from collections import OrderedDict
import torch
torch.manual_seed(0)
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
from tqdm import tqdm
from abcd.local.paths import output_path
from abcd.data.read_data import get_subjects_events_sf, subject_cols_to_events
import abcd.data.VARS as VARS
from abcd.data.define_splits import SITES, save_restore_sex_fmri_splits
from abcd.data.divide_with_splits import divide_events_by_splits
from abcd.data.var_tailoring.normalization import normalize_var
from abcd.data.pytorch.get_dataset import PandasDataset

#regresssion-specific imports
from abcd.models.regression.MLPRegressor import MLPRegressor, LinearRegressor, MLPRegressorCustom
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from abcd.training.RegressorTrainer import RegressorTrainer

#plotting
import matplotlib.pyplot as plt
import pygal
from abcd.plotting.pygal.rendering import display_html
from sklearn.metrics import confusion_matrix
import seaborn as sns
import sklearn.metrics as metrics
from sklearn.metrics import confusion_matrix
from abcd.plotting.seaborn.confusion_matrix import plot_confusion_matrix

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
from datetime import datetime

In [4]:
from abcd.analysis.regression import preprocess, train_model

In [5]:
bucketing_scheme = "sex"

In [6]:
# Determine device for training (TODO: figure out why doesn't work with mps)
device = "cpu" #("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
print("Using {} device".format(device))

Using cpu device


In [7]:
dataloaders, events_train, events_id_test, events_ood_test, feature_cols = preprocess('kbi_sex_assigned_at_birth', ['fmri', 'smri'], ood_site_num=0)

There are 9632 subjects and 18808 visits with imaging
Leaving baseline visits, we have 9085 events

There are 9085 visits after adding the target and removing NAs

Created splits
Nr. events: 7063 train, 1738 val, 284 test
Normalized targets with respect to min and max of training set. New target column containing normalized data: kbi_sex_assigned_at_birth_norm

Created dataloaders
Shape and datatype of X: torch.Size([64, 453]), torch.float32
Shape and datatype of y: torch.Size([64]), torch.float32


/Users/carolinezanze/Desktop/ABCD5/abcd/data/var_tailoring/normalization.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[y_new_name] = (df[y]-y_min)/(y_max-y_min)
/Users/carolinezanze/Desktop/ABCD5/abcd/data/var_tailoring/normalization.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[y_new_name] = (df[y]-y_min)/(y_max-y_min)
/Users/carolinezanze/Desktop/ABCD5/abcd/data/var_tailoring/normalization.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.ins

In [8]:
events_train.head()

,src_subject_id,interview_date,eventname,interview_age,rsfmri_c_ngd_ad_ngd_ad,rsfmri_c_ngd_ad_ngd_cgc,rsfmri_c_ngd_ad_ngd_ca,rsfmri_c_ngd_ad_ngd_dt,rsfmri_c_ngd_ad_ngd_dla,rsfmri_c_ngd_ad_ngd_fo,...,smri_t2ww02_cdk_smrh,smri_t2ww02_cdk_fpolerh,smri_t2ww02_cdk_tmpolerh,smri_t2ww02_cdk_tvtmrh,smri_t2ww02_cdk_insularh,smri_t2ww02_cdk_meanlh,smri_t2ww02_cdk_meanrh,smri_t2ww02_cdk_mean,kbi_sex_assigned_at_birth,kbi_sex_assigned_at_birth_norm
0,NDAR_INV003RTV85,10/01/2018,baseline_year_1_arm_1,131.0,0.647469,0.458583,0.507141,0.349996,0.575188,0.356426,...,127.719801,133.343722,127.464053,131.781973,135.007102,130.321732,129.259147,129.793951,2.0,1.0
4,NDAR_INV00J52GPG,09/05/2018,baseline_year_1_arm_1,110.0,0.415813,0.345856,0.426812,0.419386,0.406527,0.307107,...,129.631017,130.780832,131.304128,130.122385,137.059064,131.338717,130.737556,131.039779,1.0,0.0
5,NDAR_INV00LH735Y,01/29/2018,baseline_year_1_arm_1,109.0,0.581733,0.246580,0.419327,0.437370,0.440676,0.269801,...,134.650803,138.958301,130.450556,134.234003,143.198841,136.358482,135.535478,135.945715,1.0,0.0
6,NDAR_INV00LJVZK2,08/19/2017,baseline_year_1_arm_1,121.0,0.328469,0.336376,0.552241,0.659492,0.425740,0.462350,...,136.554975,133.116768,141.297444,141.343225,145.048321,138.061447,139.400439,138.732184,1.0,0.0
7,NDAR_INV00R4TXET,04/10/2018,baseline_year_1_arm_1,114.0,0.529974,0.393989,0.508073,0.539767,0.377943,0.413167,...,127.580653,136.976903,130.274707,125.604393,135.417724,128.301393,127.861643,128.082384,2.0,1.0


# Linear Regression Baseline

In [9]:
#modify
config = {'target_col': 'kbi_sex_assigned_at_birth',
          'features': ['fmri', 'smri'],
          'model': ['abcd.models.regression.MLPRegressor', 'LinearRegressor'],
          'lr': 1e-2,
          'batch_size': 64,
          'nr_epochs': 150
        }

#leave unmodified
experiment_title = 'ABCD_sex_' + config['model'][1] + "_" + datetime.now().strftime("%Y-%m-%d %H:%M:%S") #for saving results
models_path = os.path.join(output_path, experiment_title, 'models')

In [10]:
model = LinearRegressor(save_path=models_path, input_size=len(feature_cols)) #modfiy

model = model.to(device)
print(model)
trainer = train_model(model, device, config, experiment_title, dataloaders, verbose=True, bucketing_scheme=bucketing_scheme)
best_model_details = trainer.export_best_model(config=config)

LinearRegressor(
  (sigmoid): Sigmoid()
  (linear1): Linear(in_features=453, out_features=1, bias=True)
  (linear_layers): Linear(in_features=453, out_features=1, bias=True)
)


  0%|          | 0/150 [00:00<?, ?it/s]

Epoch 0
train MSELoss: 1.033 MSE: 1.033 MAE: 0.882 accuracy: 0.998
val MSELoss: 1.029 MSE: 1.027 MAE: 0.879 accuracy: 0.997
test MSELoss: 1.061 MSE: 1.052 MAE: 0.893 accuracy: 1.000

New best model.

Saved PyTorch model state LinearRegressor_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-29 01:34:15/models
Saved trainer state RegressorTrainer_optimizer_epoch0.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-29 01:34:15/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-29 01:34:15/results


  1%|          | 1/150 [00:00<00:33,  4.47it/s]

Ending epoch 1, loss 0.4878219532537031


...

 95%|█████████▌| 143/150 [00:09<00:00, 15.21it/s]

Epoch 140
train MSELoss: 0.481 MSE: 0.483 MAE: 0.483 accuracy: 1.000
val MSELoss: 0.480 MSE: 0.480 MAE: 0.480 accuracy: 1.000
test MSELoss: 0.498 MSE: 0.493 MAE: 0.493 accuracy: 1.000
Ending epoch 141, loss 0.4827408930202862


100%|██████████| 150/150 [00:10<00:00, 14.98it/s]


Finished training
Epoch 150
train MSELoss: 0.483 MSE: 0.483 MAE: 0.483 accuracy: 1.000
val MSELoss: 0.480 MSE: 0.480 MAE: 0.480 accuracy: 1.000
test MSELoss: 0.498 MSE: 0.493 MAE: 0.493 accuracy: 1.000
Saved PyTorch model state LinearRegressor_epoch150.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-29 01:34:15/models
Saved trainer state RegressorTrainer_optimizer_epoch150.pth in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-29 01:34:15/results/states
Progress stored in /Users/carolinezanze/Desktop/abcd5_output/ABCD_sex_LinearRegressor_2023-08-29 01:34:15/results


ValueError: Shape of passed values is (1, 1), indices imply (1, 2)

# Custom MLP

In [ ]:
#modify
experiment_title = 'sex_MLPReg_' + datetime.now().strftime("%Y-%m-%d %H:%M:%S") #for saving results
print(experiment_title)

config = {'target_col': 'kbi_sex_assigned_at_birth',
          'features': ['fmri', 'smri'],
          'model': ['abcd.models.regression.MLPRegressor', 'MLPRegressorCustom'],
          'batch_size': 64,

          #tune
          'lr': 1e-3,
          'nr_epochs': 150,
          'hidden_sizes': [256, 64]
        }

#leave unmodified
models_path = os.path.join(output_path, experiment_title, 'models')

In [ ]:
model = MLPRegressorCustom(save_path=models_path, input_size=len(feature_cols), hidden_sizes=config['hidden_sizes']) #modfiy

model = model.to(device)
print(model)
trainer = train_model(model, device, config, experiment_title, dataloaders, verbose=True, bucketing_scheme=bucketing_scheme)
trainer.export_best_model(config=config)

# Hyperparameter Search with Cusom MLP

In [ ]:
hidden_sizes = [
    (128, 64),
    (256, 128, 64), 
    (512, 256, 128, 64),
]

learning_rates = [1e-3, 1e-5, 1e-7]

In [ ]:
experiments = {}
global_best_val_mse = float('inf')
best_model_experiment_name = None
best_model = None

for i,learning_rate in enumerate(learning_rates):
    for j,sizes in enumerate(hidden_sizes):
        experiment_title = 'sex_MLPReg_' + datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        experiment_num = i*len(hidden_sizes) + j
        print("experiment", experiment_num, ":", experiment_title)

        config = {'target_col': 'kbi_sex_assigned_at_birth',
          'features': ['fmri', 'smri'],
          'model': ['abcd.models.regression.MLPRegressor', 'MLPRegressorCustom'],
          'batch_size': 64,
          'nr_epochs': 250,

          #tune
          'lr': 1e-3,
          'hidden_sizes': [256, 64]
        }

        config['hidden_sizes'] = sizes
        config['lr'] = learning_rate

        #define and train model
        models_path = os.path.join(output_path, experiment_title, 'models')
        model = MLPRegressorCustom(save_path=models_path, input_size=len(feature_cols), hidden_sizes=config['hidden_sizes']) #modfiy
        model = model.to(device)
        trainer = train_model(model, device, config, experiment_title, dataloaders, verbose=False, bucketing_scheme=bucketing_scheme)
        details = trainer.export_best_model(config=config)
        
        #update best model
        local_best_val_mse = details['metrics']['val']['MSE']
        if local_best_val_mse < global_best_val_mse:
            global_best_val_mse = local_best_val_mse
            best_model = details
            best_model_experiment_name = experiment_title

        #save experiment
        experiments[experiment_title] = details

print("\n\nExperiment over. Best model:", best_model_experiment_name)